In [59]:
#Import the required libraries
import pandas as pd
import numpy as np
import requests
import urllib.request
from bs4 import BeautifulSoup

In [60]:
# Url from wiki
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the html to the variable 'page'
page = urllib.request.urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page)

tblBorough = soup.find('table',{'class':'wikitable sortable'})
rows = tblBorough.findAll('tr')

In [61]:
#Create lists for storage
PostalCode=[]
Borough=[]
Neighborhood=[]

for row in rows:
    cells = row.findAll('td') 
    if len(cells)==3: #Only extract table body not heading
        PostalCode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True))   
        
df=pd.DataFrame(PostalCode,columns=['PostalCode'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood


In [62]:
# drop the rows with Borough not assigned
df = df[df.Borough != 'Not assigned']

In [63]:
#If the Neighbourhood value is not assigned, assign borough value to it
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned\n', df.Neighborhood)

#Join the neighbourhoods under the same borough
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()

df.shape

(103, 3)

# Tried with Geocoder package but is not responding somehow, so made use of provided csv

In [64]:
# # %conda install -c conda-forge geocoder 

# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}'.format('M5G'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [65]:
# Reading the csv provided for Geolocation
lat_lng_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [66]:
df = df.join(lat_lng_df.set_index('Postal Code'), on='PostalCode')

# Exploring and plotting the neighborhood under 'West Toronto'

In [67]:
toronto_data = df[df['Borough'] == 'West Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village,Dufferin\n",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton\n,Exhibition Place,Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park,The Junction South\n",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325


In [68]:
from geopy.geocoders import Nominatim
address = 'West Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Toronto are 43.653963, -79.387207.


In [69]:
import folium
# create map of West toronto using latitude and longitude values
map_west_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_west_toronto)  
    
map_west_toronto